In [2]:
import tensorflow as tf
import numpy as np

amplifier = 1
threshold_percentage = 28 # 100 - 100 * (1 - 0.2) * (1 - 0.1) 

# Define the CustomAdam optimizer class
from tensorflow.keras.optimizers import Adam

@tf.keras.utils.register_keras_serializable()
class CustomAdam(Adam):
    pass

# Load the pre-trained model
model = tf.keras.models.load_model('MAG_MNIST_vgg16_0&1_2&3_20%.h5', custom_objects={'CustomAdam': CustomAdam})

# Load the additional pre-trained model for weight multiplication
progress_model = tf.keras.models.load_model('freezing_value_progress_MNIST_0&1_2&3.h5', custom_objects={'CustomAdam': CustomAdam})

# Copy the structure and parameters of the model
freezing_value_mnist = tf.keras.models.clone_model(model)
freezing_value_mnist.set_weights(model.get_weights())

# Multiply the weights of the cloned model with the weights of the progress model
for layer, progress_layer in zip(freezing_value_mnist.layers, progress_model.layers):
    if len(layer.get_weights()) > 0:
        weights = layer.get_weights()
        progress_weights = progress_layer.get_weights()
        new_weights = []

        for weight_matrix, progress_weight_matrix in zip(weights, progress_weights):
            new_weight_matrix = weight_matrix * progress_weight_matrix
            new_weights.append(new_weight_matrix)

        # Set the new weights
        layer.set_weights(new_weights)

# Modify the weights as specified
for layer in freezing_value_mnist.layers:
    if len(layer.get_weights()) > 0:
        weights = layer.get_weights()
        new_weights = []

        for weight_matrix in weights:
            # Compute the biggest weight magnitude in the whole layer
            max_magnitude = np.max(np.abs(weight_matrix))
            
            # Apply the transformation to each weight
            new_weight_matrix = 1 - (np.abs(weight_matrix) / max_magnitude) ** amplifier
            new_weights.append(new_weight_matrix)

        # Set the new weights
        layer.set_weights(new_weights)

# Further modify the weights by setting the smallest specified percentage weights for each layer to 0
for layer in freezing_value_mnist.layers:
    if len(layer.get_weights()) > 0:
        weights = layer.get_weights()
        new_weights = []

        for weight_matrix in weights:
            # Flatten the weight matrix and sort the weights by magnitude
            flat_weights = weight_matrix.flatten()
            threshold = np.percentile(np.abs(flat_weights), threshold_percentage)
            
            # Set the smallest weights to 0
            new_weight_matrix = np.where(np.abs(weight_matrix) < threshold, 0, weight_matrix)
            new_weights.append(new_weight_matrix)

        # Set the new weights
        layer.set_weights(new_weights)

# Print all the weights in the model
for layer in freezing_value_mnist.layers:
    if len(layer.get_weights()) > 0:
        weights = layer.get_weights()
        print(f"Weights for layer {layer.name}:")
        for weight_matrix in weights:
            print(weight_matrix)

# Save the modified model
freezing_value_mnist.save('freezing_value_mag_0&1_2&3_10%_^1.h5')

print("Model 'freezing_value_mag_0&1_2&3_10%_^1.h5' has been saved.")


2024-08-13 01:39:10.664704: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-13 01:39:10.681889: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 01:39:10.701211: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 01:39:10.707098: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-13 01:39:10.721120: I tensorflow/core/platform/cpu_feature_guar

Weights for layer conv2d:
[[[[0.7177397  1.         0.         ... 0.         0.37276882
    0.99364126]
   [0.5357325  0.         0.         ... 0.         1.
    0.7382399 ]
   [0.8613992  0.         0.98980254 ... 0.46622217 0.88515764
    0.99356854]]

  [[0.9702296  0.7461306  0.45309603 ... 0.41051954 0.85863435
    1.        ]
   [1.         0.         0.         ... 0.7232842  0.39239573
    0.8634616 ]
   [1.         0.72295845 0.795831   ... 0.9176273  0.
    0.6987746 ]]

  [[0.53180695 0.88812065 1.         ... 1.         1.
    0.5760021 ]
   [0.         1.         0.         ... 0.         0.
    0.6339765 ]
   [0.         0.7727414  0.91803545 ... 0.7386147  0.5207928
    1.        ]]]


 [[[0.8951084  0.6201109  0.         ... 0.5312693  1.
    0.4414361 ]
   [1.         0.9131583  0.         ... 0.9124658  1.
    0.861888  ]
   [0.         1.         0.8346237  ... 0.5705128  0.8064153
    0.        ]]

  [[0.6349914  0.         1.         ... 1.         0.72078204
   

Weights for layer dense:
[[1.         0.9344281  0.9789817  ... 0.         0.8788542  0.61864305]
 [0.89168435 0.92296505 1.         ... 0.7811732  0.86203504 0.7734059 ]
 [0.53838885 0.6729278  0.8126098  ... 0.97667295 0.56862855 0.61219406]
 ...
 [0.64896977 0.         0.         ... 0.9901207  0.63560295 0.529901  ]
 [0.         0.76275426 0.7628342  ... 0.         0.         1.        ]
 [0.         0.         0.6790682  ... 0.69907165 0.83667886 0.74782026]]
[0.         0.84261554 1.         ... 0.9002429  1.         1.        ]
Weights for layer dense_1:
[[0.7572897  0.         0.79550916 ... 0.8078327  1.         0.        ]
 [0.         0.         0.         ... 1.         0.8006639  1.        ]
 [1.         0.         1.         ... 0.         0.         0.90750176]
 ...
 [0.5143304  1.         0.53571016 ... 0.51773036 0.         0.5311029 ]
 [0.50692755 0.51183474 0.         ... 0.         0.         0.        ]
 [0.6668274  0.8360224  1.         ... 0.5373944  1.         0